In [1]:
import os
home = r"C:\Users\rose_\OneDrive\Desktop\DA_Project\DA_Project"   ## use your direcitory (.../data)
home2 = home + '/data'  ## /data/data path can get access to all raw data (tweets, followings)
os.chdir(home)

In [2]:
import tweepy
### Set an API
with open("twitter_key&secret key.txt") as f:
    api_key = f.read().split("\n")
consumer_key = api_key[0]
consumer_secret = api_key[1]
access_token = api_key[2]
access_token_secret = api_key[3]

import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Setting wait_on_rate_limit and wait_on_rate_limit_notify to True makes the API object print a message and wait if the rate limit is exceeded
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

<h2> Preprocessing

In [21]:
def text_preprocessing(id_list):
    from tqdm import tqdm_notebook
    # Read tweet into dictionary
    import os
    path = r"C:\Users\rose_\OneDrive\Desktop\DA_Project\DA_Project\data" 
    os.chdir(path)
    tweet = dict()
    description = dict()
    for i in tqdm_notebook(id_list):
        path2 = path +f"/{i}"
        files = os.listdir(path2)
        for file in files:
            if file=='description.txt': 
                f = open(path2+"/"+file); 
                iter_f = iter(f);
                str = ""
                for line in iter_f: 
                    str = str + line
                description[i]=str
            elif file=='tweet.txt': 
                f = open(path2+"/"+file); 
                iter_f = iter(f); 
                str = ""
                for line in iter_f: 
                    str = str + line
                tweet[i]=str

    # Text cleaning, get rid of non-English text
    from langdetect import detect
    import re
    import demoji

    tweet_cleaned=dict()
    description_cleaned=dict()

    for i in tqdm_notebook(tweet.keys()):
        string_=tweet[i]
        # remove @name
        string_=re.sub(r'RT @\S+', '', string_, count=0, flags=0)

        # remove 'https://t.co/\w+'
        string_=re.sub(r'https://t.co/\w+', '', string_, count=0, flags=0)

        #count the number of different emojis of each user
        count=dict()
        try:
            for j in demoji.findall(tweet[i]).keys():
                count[j]=len(re.findall(j,tweet[i]))
        except:
            count['None']='No emoji'

        # remove emoji
        string_=demoji.replace(string_,'')

        # remove other languages and user with empty text
        if not string_=='':
            try:
                if detect(string_)=='en' and string_!=None:
                    # clean again
                    string_=re.sub(r'[^A-Za-z0-9 _.,!\"\'|]+'," ",string_)
                    tweet_cleaned[i]=string_
                    description_cleaned[i]=description[i]
            except:
                print("We can not detect the language")
    keys = [int(i) for i in list(tweet_cleaned.keys())]
    
    return tweet, tweet_cleaned, keys

In [ ]:
valid_data = pd.read_csv('full_data.csv')
valid_id = list(valid_data['id'])
tweet, tweet_cleaned, keys = text_preprocessing(valid_id)
id_cleaned = pd.DataFrame(keys,columns=["id"])
# Delete non-english users in dataframe
valid_data = valid_data.merge(id_cleaned,how="right",on="id")
valid_data.head()

<h2>EMOJI

In [ ]:
# get the all the emoji for each one
import demoji
from tqdm import tqdm_notebook
emoji={}
for i in tqdm_notebook(tweet.keys()):
    emoji[i]=demoji.findall(tweet[i])
 
emoji_count=dict()
import re
for i in tqdm_notebook(tweet.keys()):
    count=list()
    try:
        for j in emoji[i].keys():
            count.append([j,len(re.findall(j,tweet[i]))])
    except:
        pass
    emoji_count[i]=count
emoji_count